In [102]:
import pandas as pd
import numpy as np
import mysql.connector
from sqlalchemy import create_engine
import sqlalchemy
from sqlalchemy import *
from census import Census
from MySQLconfig import username, password

# Census API Key
from config import api_key
c = Census(api_key, year=2017)

# Pull various datasources

In [103]:
#Dataset of bars
#df = pd.read_csv('https://query.data.world/s/7c4vc4flpfmphqsmsjhbd55fsj3w32')
df = pd.read_csv('barsinfo.csv')

In [104]:
df.head()

,id,address,categories,city,country,hours,keys,latitude,longitude,menus,name,postalCode,province,twitter,websites
0,AVweN1T_ByjofQCxwSPL,Syr Airport,Brewery,Syracuse,US,NaN,us/ny/syracuse/syrairport/2098450797,NaN,NaN,NaN,Adirondack Pub,NaN,NY,NaN,NaN
1,AVwdkF9ZIN2L1WUfxDfw,1320 N Doctor Martin Luther King Dr,Office and Cafeteria Brewer's Hill,Milwaukee,US,NaN,us/wi/milwaukee/1320ndoctormartinlutherkingdr/...,NaN,NaN,NaN,The Waterfront Cafe,53212,WI,NaN,NaN
2,AVwdlPkf_7pvs4fz7TMr,608 4th St,Brewery Downtown Clovis,Clovis,US,"[{""day"":""Wed - Sat"",""hour"":""5:00 PM - 8:00 PM""}]",us/ca/clovis/6084thst/-932885010,NaN,NaN,NaN,559 Local Brewing,93612,CA,NaN,http://fresnobrewing.com
3,AVwcucY0kufWRAb5yWbU,1103 Mills St,breweries & brew pubs,Black Earth,US,NaN,us/wi/blackearth/1103millsst/-757413411,43.138995,-89.747904,NaN,Shack The,53515-9454,WI,NaN,NaN
4,AVweCkd5kufWRAb5_eaI,130 W Riverside Dr,Brewery and Gastropub Downtown Jeffersonville,Jeffersonville,US,"[{""day"":""Wed - Thu"",""hour"":""3:00 PM - 9:00 PM""...",us/utica/jeffersonville/130wriversidedr/787280564,NaN,NaN,NaN,Flat12 Bierworks,47130,Utica,flat12bierwerks,http://flat12.me/brewery


In [105]:
#Capture columns needed and rename
bars_columns = ['postalCode','name'] #picks the column 
dfBars = df[bars_columns].copy()
dfBars=dfBars.rename(columns={'postalCode':'zipcode', 'name':'bar_name'}) #rename columns

dfBars.head()

,zipcode,bar_name
0,NaN,Adirondack Pub
1,53212,The Waterfront Cafe
2,93612,559 Local Brewing
3,53515-9454,Shack The
4,47130,Flat12 Bierworks


In [106]:
#Clean data and reformat zipcode
dfBars.dropna(subset=['zipcode'],inplace=True) # Drop rows with no zip code listed
dfBars['zipcode'] = dfBars['zipcode'].str.zfill(5) # Prepend '0' if zip code less than 5 digits long
dfBars['zipcode'] = dfBars['zipcode'].str.slice(start=0,stop=5) # Keep only first 5 digits of zip code
dfBars.head()

,zipcode,bar_name
1,53212,The Waterfront Cafe
2,93612,559 Local Brewing
3,53515,Shack The
4,47130,Flat12 Bierworks
5,98072,Redhook Ale Brewery Inc


In [107]:
#Group bars by zipcode to get count
groupedbars = dfBars.groupby(['zipcode'], as_index=False).count()
groupedbars.sort_values('bar_name', ascending=False, inplace=True)
groupedbars=groupedbars.rename(columns={'bar_name':'bar_count'}) #rename columns


groupedbars.head()


,zipcode,bar_count
263,04412,57
5362,93446,51
5479,94558,49
5604,95448,44
2659,43215,42


In [108]:
#Import zipcode information
zipdf = pd.read_csv('zip_code_database.csv')
zipdf.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [109]:
#Reformat zip column
zipdf['zip'].apply(str)
zipdf['zip']=zipdf['zip'].apply(str)
zipdf.dropna(subset=['zip'],inplace=True) # Drop rows with no zip code listed
zipdf['zip'] = zipdf['zip'].str.zfill(5) # Prepend '0' if zip code less than 5 digits long

zipdf.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
0,00501,UNIQUE,0,Holtsville,NaN,I R S Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,562
1,00544,UNIQUE,0,Holtsville,NaN,Irs Service Center,NY,Suffolk County,America/New_York,631,NaN,US,40.81,-73.04,0
2,00601,STANDARD,0,Adjuntas,NaN,"Colinas Del Gigante, Jard De Adjuntas, Urb San...",PR,Adjuntas Municipio,America/Puerto_Rico,"787,939",NaN,US,18.16,-66.72,0
3,00602,STANDARD,0,Aguada,NaN,"Alts De Aguada, Bo Guaniquilla, Comunidad Las ...",PR,Aguada Municipio,America/Puerto_Rico,"787,939",NaN,US,18.38,-67.18,0
4,00603,STANDARD,0,Aguadilla,Ramey,"Bda Caban, Bda Esteves, Bo Borinquen, Bo Ceiba...",PR,Aguadilla Municipio,America/Puerto_Rico,787,NaN,US,18.43,-67.15,0


In [110]:
#zipdf.info()

In [137]:
#Pick columns from zipdf
zip_columns = ['zip','primary_city','state','county','irs_estimated_population_2015'] #picks the column 
zip_info = zipdf[zip_columns].copy()
zip_info=zip_info.rename(columns={'zip':'zipcode','irs_estimated_population_2015':'irs_est_population_2015'}) #rename columns

zip_info.head()

,zipcode,primary_city,state,county,irs_est_population_2015
0,00501,Holtsville,NY,Suffolk County,562
1,00544,Holtsville,NY,Suffolk County,0
2,00601,Adjuntas,PR,Adjuntas Municipio,0
3,00602,Aguada,PR,Aguada Municipio,0
4,00603,Aguadilla,PR,Aguadilla Municipio,0


# Census Data pull

In [112]:
# Run Census Search to retrieve data on all zip codes (2017 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E","B23025_004E","B01002_001E",
                          "B19301_001E"), {'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "NAME":"Name", "zip code tabulation area": "Zipcode"})

# Final DataFrame
census2017 = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income"]]



In [113]:
#Display census data and rename columns
census2017=census2017.rename(columns={'Zipcode':'zipcode','Median Age':'MedianAge','Household Income':'HouseholdIncome','Per Capita Income':'PerCapitaIncome'}) #rename columns
census2017.head()

,zipcode,Population,MedianAge,HouseholdIncome,PerCapitaIncome
0,00601,17599.0,38.9,11757.0,7041.0
1,00602,39209.0,40.9,16190.0,8978.0
2,00603,50135.0,40.4,16645.0,10897.0
3,00606,6304.0,42.8,13387.0,5960.0
4,00610,27590.0,41.4,18741.0,9266.0


In [114]:
#Remove Nans
census2017.dropna()

,zipcode,Population,MedianAge,HouseholdIncome,PerCapitaIncome
0,00601,17599.0,38.9,11757.0,7041.0
1,00602,39209.0,40.9,16190.0,8978.0
2,00603,50135.0,40.4,16645.0,10897.0
3,00606,6304.0,42.8,13387.0,5960.0
4,00610,27590.0,41.4,18741.0,9266.0
5,00612,62566.0,41.4,17744.0,10283.0
6,00616,10687.0,43.7,14918.0,9002.0
7,00617,24508.0,38.7,17157.0,8762.0
8,00622,7405.0,48.4,16727.0,11935.0
9,00623,42321.0,41.3,16401.0,9101.0


In [115]:
#Remove bad values
census2017_1=census2017[census2017.MedianAge !=-666666666.0]
#census2017[census2017.HouseholdIncome !=-666666666.0]
#census2017[census2017.PerCapitaIncome !=-666666666.0]



In [116]:
#Remove bad values
census2017_2=census2017_1[census2017_1.HouseholdIncome !=-666666666.0]

In [117]:
#Remove bad values
census2017_cleaned=census2017_2[census2017_2.PerCapitaIncome !=-666666666.0]

In [118]:
#Export cleaned file for reference
#census2017_cleaned.to_csv("census_data_2017_cleaned.csv", encoding="utf-8", index=False)

In [139]:
#Merge bars and census. 
bars_census = pd.merge(census2017_cleaned, groupedbars, on='zipcode', how='left')

#Clean and perform calculations
bars_census.dropna(subset=['bar_count'],inplace=True) # Drop rows with no bars
bars_census['Bars_per_Capita']=bars_census['bar_count']/bars_census['Population']
bars_census['Bars_per_Income']=bars_census['bar_count']/bars_census['HouseholdIncome']
#bars_census.info()


In [120]:
#Export cleaned file for reference
#bars_cleaned.to_csv("bars_cleaned.csv", encoding="utf-8", index=False)

In [121]:
#Remove bad values
#bars_clean=bars_census[bars_census.MedianAge !=-666666666.0]
#bars_census[bars_census.HouseholdIncome !=-666666666.0]
#bars_census[bars_census.PerCapitaIncome !=-666666666.0]


In [122]:
#Remove bad values
#bars_clean2=bars_clean[bars_clean.HouseholdIncome !=-666666666.0]

In [123]:
#Remove bad values
#bars_cleaned=bars_clean2[bars_clean2.PerCapitaIncome !=-666666666.0]

In [124]:
#bars_cleaned.head()

# MySQL - loading information to MySQL database

In [180]:
#MySQL Connection

rds_connection_string = username + ":" + password +"@127.0.0.1/bars_db"

engine = create_engine(f'mysql://{rds_connection_string}')

In [181]:
#Show available tables
engine.table_names()

['bars_count', 'bars_values', 'census', 'zip_info']

In [182]:
#Clear existing data in tables before loading information
engine.execute("truncate table bars_count")
engine.execute("truncate table bars_values")
engine.execute("truncate table census")
engine.execute("truncate table zip_info")

In [183]:
#Loads Count of Bars by zip code

groupedbars.to_sql(name='bars_count', con=engine, if_exists='append', index=False)
#pd.read_sql_query('select * from bars_count', con=engine).head()

In [184]:
#Loads Bars data with census
bars_census.to_sql(name='bars_values', con=engine, if_exists='append', index=False)

In [185]:
#Loads Census data
census2017_cleaned.to_sql(name='census', con=engine, if_exists='append', index=False)


In [186]:
#Loads Zip Info
zip_info.to_sql(name='zip_info', con=engine, if_exists='append', index=False)